In [ ]:

# builtins.__dict__.update(locals())
# t = timeit.Timer('conved = list(map(lambda x: list(map(lambda y: y.value[0], x)), prevObs))')
# time = t.timeit(10000)
# print(time / 10000)
import faulthandler
import multiprocessing
# logger = open('log.txt', 'w')
# faulthandler.enable(file=logger)

import tensorflow as tf
from tensorflow.keras.layers import InputLayer, Conv2D, BatchNormalization, LeakyReLU, Add, Flatten, Dense, Concatenate, Dot, Reshape
print(tf.__version__)

import tensorflow.keras.backend as K
# tf.autograph.set_verbosity(10,alsologtostdout=True)

import numpy as np

import gym
import gym_battleship1

import builtins
# import timeit # DEBUG Only
import time
# from collections import deques



print(tf.__version__)

In [ ]:

# MODEL TWEAKS
NUM_GAMES = 5100
FILTERS = 64 # 64 because its cool
EPSILON = 0.0 # Epsilon must start close to one or model training will scew incredibelly
LEARNING_RATE = 0.001
MOMENTUM = 0.1
CHANNEL_TYPE = "channels_first"

In [ ]:
def convLayerCluster(inp):
	m = Conv2D(filters=FILTERS,kernel_size=(3,3),padding="same",use_bias=False,activation='linear',kernel_regularizer=reg,data_format=CHANNEL_TYPE)(inp)
	m = BatchNormalization(axis=1)(m)
	return LeakyReLU()(m)

def residualLayerCluster(inp):
	m = convLayerCluster(inp)
	m = Conv2D(filters=FILTERS,kernel_size=(3,3),padding="same",use_bias=False,activation='linear',kernel_regularizer=reg,data_format=CHANNEL_TYPE)(m)
	m = BatchNormalization(axis=1)(m)
	m = Add()([inp,m])
	return LeakyReLU()(m)

# @tf.function
def customLoss(y_true, y_pred):
	crossEnt = tf.nn.sigmoid_cross_entropy_with_logits(labels=y_true,logits=y_pred)
	return tf.reduce_mean(crossEnt)

# def softmax_cross_entropy_with_logits(y_true, y_pred):

#BUILDING MODEL
# reg = tf.keras.regularizers.L2(l2=0.0001)
reg = None # see if no reg helps 
def buildModel():
	inputLay = tf.keras.Input(shape=(6,10,10))#12

	m = Conv2D(filters=FILTERS,kernel_size=(3,3),padding="same",use_bias=False,activation='linear',kernel_regularizer=reg,data_format=CHANNEL_TYPE)(inputLay)
	# m = Conv2D(64,3,data_format=CHANNEL_TYPE)(inputLay)
	m = BatchNormalization(axis=1)(m) #-1 for channels last
	m = LeakyReLU()(m)

	m = residualLayerCluster(m)
	m = residualLayerCluster(m)
	# m = residualLayerCluster(m) # removed on cluster for added simplricity

	m = Conv2D(filters=6,kernel_size=(1,1),padding="same",use_bias=False,activation='linear',kernel_regularizer=reg,data_format=CHANNEL_TYPE)(m) #12
	m = BatchNormalization(axis=1)(m)
	m = LeakyReLU()(m)

	m = Flatten()(m)
	og = Flatten()(inputLay)
	r = Concatenate(axis=1)([m,og])
	out = Dense(100)(r) #activation='sigmoid'

	return tf.keras.Model(inputs=inputLay, outputs=out)

def oldBuildModel():
	inputLay = tf.keras.Input(shape=(6,10,10))
	c1 = Conv2D(1, (1, 1), data_format="channels_first")(inputLay)
	c2 = Conv2D(1, (1, 1), data_format="channels_first")(inputLay)
	f1 = Reshape((1,100))(c1)
	f2 = Reshape((1,100))(c2)
	d = Dot(axes=1)([f1,f2])
	f3 = Flatten(data_format="channels_first")(d)
	# inputLay = InputLayer(input_shape=(10,10,6))
	# out = Conv2D
	out = Dense(100)(f3) #INCLUDE
	return tf.keras.Model(inputs=inputLay, outputs=out)

model = buildModel()
# model = tf.keras.models.load_model('saved_model/my_model',compile=False)

# lossFunc = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
optim = tf.keras.optimizers.SGD(lr=LEARNING_RATE, momentum = MOMENTUM) # optim = tf.keras.optimizers.SGD(lr=LEARNING_RATE, momentum = MOMENTUM)
error = tf.keras.metrics.MeanAbsoluteError()
lossAvg = tf.keras.metrics.Mean()
# error = tf.keras.metrics.Mean()
accuracy = tf.keras.metrics.CategoricalAccuracy()
gameLength = tf.keras.metrics.Mean()
model.compile(optimizer=optim,loss=customLoss,metrics=[error,accuracy])
summary_writer = tf.summary.create_file_writer('logs')
print(model.summary())
# model.load_weights('saved_model/checkpoints/cp')

In [ ]:
# Globals
ct = time.time()
env = gym.make('battleship1-v1')

@tf.function # Decoration is 10 fold faster
def makeMove(obs,e):
	# print("Traced with " + str(e))
	if e > 0:
		r = tf.random.uniform(shape=[],dtype=tf.dtypes.float16)
		if r < EPSILON:
			return tf.random.uniform(shape=[],maxval=100,dtype=tf.dtypes.int64)
		else:
			logits = model.predict_step(obs)
			return tf.argmax(logits, 1)[0]
	logits = model.predict_step(obs)
	return tf.argmax(logits, 1)[0]

@tf.function
def trainGrads(feature,expect):
	with tf.GradientTape() as tape:
		# predictions = self.model(features)
		logits = model(feature,training=True)
		loss = customLoss(expect, logits[0])
		# loss = lossFunc(expect, predictions[0])
	gradients = tape.gradient(loss, model.trainable_variables)
	optim.apply_gradients(zip(gradients, model.trainable_variables))
	error.update_state(expect, logits[0])
	accuracy.update_state(expect, logits)
	lossAvg.update_state(loss)
	return gradients

# def singleStepConv():
hits = 0
iterartions = 0
# sess = tf.Session()
for epoch in range(0,NUM_GAMES):
	prevObs = env.reset()
	prevObs = [[[x.value[0] for x in y] for y in c] for c in prevObs] # redo timeit with numpy
	# prevObs = tf.reshape(tf.transpose(tf.convert_to_tensor(prevObs)),shape=(1,10,10,6)) # ONLY NEEDED FOR CPUS
	prevObs = tf.convert_to_tensor([prevObs])
	# prevObs = tf.reshape(prevObs, (1,10,10,6))

	observations = [] # could also use deque
	expecteds = []
	done = False

	slotsLeft = np.ones(shape=100,dtype=np.float32)
	while not done:
		# Could Accelerate this, however few tf methods, and a couple of outside methods
		move = makeMove(prevObs,EPSILON).numpy()
		# print(move.numpy())
		obs, reward, done, out = env.step(move)
		obs = [[[x.value[0] for x in y] for y in c] for c in obs] # numpy may be faster
		out = [t.value[0] for t in out]

		# out = tf.Variable(tf.zeros([100]))

		if reward:
			hits += 1
		# 	out[move].assign(1.)
		# 	observations.append(prevObs[0])
		# 	expecteds.append(out)
		# elif done:
		# 	observations.append(prevObs[0])
		# 	expecteds.append(tf.cast(tf.convert_to_tensor(out),dtype=tf.dtypes.float32))

		observations.append(prevObs[0])
		expecteds.append(tf.convert_to_tensor(slotsLeft))
		slotsLeft[move] = 0
		# expecteds.append(tf.cast(tf.convert_to_tensor(out),dtype=tf.dtypes.float32))

		prevObs = tf.convert_to_tensor([obs])
		# prevObs = tf.reshape(tf.transpose(tf.convert_to_tensor(prevObs)),shape=(1,10,10,6))
		iterartions += 1
		if done:
			gameLength.update_state(env.counter)

	# TRAINING

	# for i in range(len(observations)): # FOR DEBUGGING #
	# 	ret = trainGrads(tf.reshape(observations[i],shape=(1,10,10,6)),expecteds[i])
	# 	pass
	
	observations = tf.stack(observations)
	expecteds = tf.stack(expecteds)
	ret = model.train_on_batch(x=observations,y=expecteds,reset_metrics=False,return_dict=True)
	lossAvg.update_state(ret['loss'])

	if (epoch+1) % (NUM_GAMES // 30) == 0:
		with summary_writer.as_default():
			tf.summary.scalar('Loss', lossAvg.result(), step=epoch+1)
			tf.summary.scalar('Error', error.result(), step=epoch+1)
			tf.summary.scalar('Accuracy', accuracy.result()*100, step=epoch+1)
			tf.summary.scalar('Hits', 100*hits / iterartions, step=epoch+1)
			tf.summary.scalar('Game Length', gameLength.result(), step=epoch+1)
		print(f"Completed {epoch+1} epochs at {round(EPSILON,7)} in {round(time.time() - ct, 3)}s. L={round(float(lossAvg.result().numpy()),6)} E={round(float(error.result().numpy()),6)} A={round(float(accuracy.result().numpy()),6)} H={round(hits / iterartions,6)} I={round(float(gameLength.result().numpy()),3)}")
		error.reset_states()
		accuracy.reset_states()
		gameLength.reset_states()
		lossAvg.reset_states()
		hits = 0
		iterartions = 0
		if EPSILON > 0.06:
			EPSILON -= 0.03
		else:
			EPSILON /= 1.75
		# model.save_weights('saved_model/checkpoints/cp')
		ct = time.time()
		# x = tf.function(makeMove)

model.save('saved_model/my_model')
print("Model Saved")